# bucket to sql


In [1]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Z86b4GtBqZ7xCJSoNUH14rtP9LrvY6&access_type=offline&code_challenge=p9HopdZlGEBotAo0GvrKVfTOyruzPXu3qiJjhfWHSN4&code_challenge_method=S256


You are now logged in as [elyesmaalel@gmail.com].
Your current project is [dauphine-437611].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
# Installation des dépendances nécessaires
%pip install -q pandas google-cloud-storage langchain-google-vertexai langchain-google-cloud-sql-pg python-dotenv

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.14 requires openai<2.0.0,>=1.58.1, but you have openai 0.28.0 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import pandas as pd
from google.cloud import storage
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_pg import PostgresEngine, PostgresVectorStore
from langchain_core.documents.base import Document
from dotenv import load_dotenv
import asyncio

In [4]:
PROJECT_ID = "my-dproject-452220"  # À modifier
BUCKET_NAME = "elyes_bucket"      # À modifier
DATABASE = "health_database"       # À modifier
INSTANCE = "elyessiki"       # À modifier
DB_USER = "postgres"           # À modifier

# Chargement des variables d'environnement pour les informations sensibles
load_dotenv()
DB_PASSWORD = os.environ["DB_PASSWORD"]

In [5]:


# Initialize a client
client = storage.Client()

bucket = client.get_bucket(BUCKET_NAME)

# List all files in the 'data' directory
blobs = bucket.list_blobs(prefix="data/")

print("Files in 'elyes-bucket':")
for blob in blobs:
    print(blob.name)

Files in 'elyes-bucket':
data/
data/medquadd.csv


In [8]:
# Specify the file path (replace with an actual file from your bucket)
file_path = 'data/medquadd.csv'

# Get the blob object
blob = bucket.get_blob(file_path)


if blob:
    print(f"Information for '{file_path}':")
    print(f"Size: {blob.size} bytes")
    print(f"Content Type: {blob.content_type}")
    formatted_updated_on = updated_on.strftime("%Y-%m-%d %H:%M:%S")  # Format: YYYY-MM-DD HH:MM:SS
    print(f"Updated On: {formatted_updated_on}")
    print(f"Blob name: {blob.name}")
else:
    print(f"File '{file_path}' not found in the bucket.")

Information for 'data/medquadd.csv':
Size: 22599845 bytes
Content Type: text/csv
Updated On: 2025-02-27 20:46:50
Blob name: data/medquadd.csv


In [12]:
from google.cloud import storage

# Initialiser le client
client = storage.Client()

# Obtenez le projet actif
project_id = client.project

# Afficher l'ID du projet
print(f"Le projet actuel actif est : {project_id}")


Le projet actuel actif est : dauphine-437611


In [13]:
from google.cloud import storage

# Changer le projet actif en initialisant le client avec l'ID du projet
client = storage.Client(project='my-dproject-452220')

# Afficher l'ID du projet pour vérifier le changement
print(f"Le projet actuel actif est : {client.project}")


Le projet actuel actif est : my-dproject-452220


In [15]:
from google.cloud import storage  # Module pour interagir avec Google Cloud Storage

# Définition du répertoire local où les fichiers seront téléchargés
DOWNLOADED_LOCAL_DIRECTORY = "./downloaded_files"

# S'assurer que le dossier 'downloaded_files' existe, sinon le créer
os.makedirs(DOWNLOADED_LOCAL_DIRECTORY, exist_ok=True)

# Initialisation du client Google Cloud Storage
client = storage.Client()

# Nom du bucket contenant le fichier (à modifier selon votre projet)
BUCKET_NAME = 'elyes_bucket'

# Récupération du bucket à partir du client Storage
bucket = client.get_bucket(BUCKET_NAME)

def download_file_from_bucket(bucket, file_path: str) -> str:
    """
    Télécharge un fichier spécifique depuis un bucket Google Cloud Storage et l'enregistre en local.

    :param bucket: Objet représentant le bucket GCS
    :param file_path: Chemin du fichier à télécharger dans le bucket (ex: 'data/medquad.csv')
    :return: Chemin du fichier téléchargé en local
    """
    
    # Créer un objet 'blob' représentant le fichier dans le bucket
    blob = bucket.blob(file_path)
    
    # Extraire uniquement le nom du fichier depuis le chemin
    local_file_name = os.path.basename(file_path)
    
    # Construire le chemin complet où le fichier sera stocké localement
    local_filepath = os.path.join(DOWNLOADED_LOCAL_DIRECTORY, local_file_name)
    
    # Télécharger le fichier depuis le bucket et l'enregistrer localement
    blob.download_to_filename(local_filepath)
    
    # Affichage d'un message de confirmation
    print(f"✅ Téléchargement terminé : '{file_path}' → '{local_filepath}'")
    
    # Retourner le chemin local du fichier téléchargé
    return local_filepath


In [ ]:
import pandas as pd
from typing import List
import os
from google.cloud import storage

# Définir un objet Document qui simule le stockage du contenu et des métadonnées
class Document:
    def __init__(self, page_content: str, metadata: dict):
        self.page_content = page_content  # Contenu du document (question)
        self.metadata = metadata          # Métadonnées associées au document

# Fonction pour convertir un fichier CSV en une liste d'objets Document
def csv_to_documents(local_filepath: str) -> List[Document]:
    """
    Cette fonction lit un fichier CSV, extrait les données pertinentes,
    et crée une liste d'objets Document à partir des questions et métadonnées.
    
    :param local_filepath: Le chemin local du fichier CSV à traiter.
    :return: Une liste d'objets Document.
    """
    
    # Charger le fichier CSV dans un DataFrame
    df = pd.read_csv(local_filepath)
    
    # Liste pour stocker les documents créés
    documents = []
    
    # Parcourir chaque ligne du DataFrame pour créer un document
    for index, row in df.iterrows():
        # Le contenu du document est la question
        content = row['question']
        
        # Les métadonnées sont extraites des autres colonnes du CSV
        metadata = {
            "answer": row['answer'],            # Réponse à la question
            "source": row['source'],            # Source de l'information
            "focus_area": row['focus_area'],    # Domaine de focus
            "row_index": index                  # Index de la ligne dans le CSV
        }
        
        # Créer un objet Document avec le contenu et les métadonnées
        doc = Document(page_content=content, metadata=metadata)
        
        # Ajouter le document à la liste
        documents.append(doc)
    
    # Retourner la liste complète des documents
    return documents

# Fonction pour télécharger un fichier depuis un bucket Google Cloud Storage
def download_file_from_bucket(bucket_name: str, file_path: str, local_directory: str) -> str:
    """
    Télécharge un fichier spécifique depuis un bucket Google Cloud Storage et l'enregistre en local.
    
    :param bucket_name: Nom du bucket dans GCS
    :param file_path: Chemin du fichier à télécharger dans le bucket
    :param local_directory: Dossier local où le fichier sera enregistré
    :return: Chemin local du fichier téléchargé
    """
    
    # Créer le répertoire local s'il n'existe pas
    os.makedirs(local_directory, exist_ok=True)
    
    # Initialiser le client Google Cloud Storage
    client = storage.Client()
    
    # Récupérer le bucket
    bucket = client.get_bucket(bucket_name)
    
    # Créer un objet blob représentant le fichier dans le bucket
    blob = bucket.blob(file_path)
    
    # Extraire le nom du fichier depuis le chemin du fichier
    local_filename = os.path.basename(file_path)
    
    # Construire le chemin complet du fichier local
    local_filepath = os.path.join(local_directory, local_filename)
    
    # Télécharger le fichier depuis le bucket
    blob.download_to_filename(local_filepath)
    
    # Retourner le chemin du fichier téléchargé
    return local_filepath

# Exemple d'utilisation 
bucket_name = 'elyes_bucket'  # Remplacez par le nom de votre bucket
file_path = "data/medquadd.csv"  # Chemin du fichier dans le bucket
local_directory = "./downloaded_files"  # Répertoire local de téléchargement

# Télécharger le fichier depuis le bucket
#local_filepath = download_file_from_bucket(bucket_name, file_path, local_directory)

# Convertir le fichier CSV en une liste d'objets Document
documents = csv_to_documents(local_filepath)

# Afficher quelques informations sur les documents créés dans le notebook
for doc in documents[:5]:  # Affiche les 5 premiers documents pour exemple
    print(f"Question: {doc.page_content}")
    print(f"Answer: {doc.metadata['answer']}")
    print(f"Source: {doc.metadata['source']}")
    print(f"Focus Area: {doc.metadata['focus_area']}")
    print(f"Row Index: {doc.metadata['row_index']}\n")


Question: what is (are) glaucoma ?
Answer: glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. while glaucoma can strike anyone, the risk is much greater for people over 60. how glaucoma develops  there are several different types of glaucoma. most of these involve the drainage system within the eye. at the front of the eye there is a small space called the anterior chamber. a clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (watch the video to learn more about glaucoma. to enlarge the video, click the brackets in the lower right-hand corner. to reduce the video, press the escape (esc) button on your keyboard.) in glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. as the fluid builds up, the pressure inside the eye rises. unless this pressure is controlled, it may cause damage to the optic nerve and other parts of the eye and result in loss of vision. open-angle g

In [20]:
# Vérification
print(f"Nombre de documents créés : {len(documents)}")
print("\nExemple du premier document :")
print(documents[0].page_content)
print("\nMétadonnées :", documents[0].metadata)

Nombre de documents créés : 16345

Exemple du premier document :
what is (are) glaucoma ?

Métadonnées : {'answer': "glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. while glaucoma can strike anyone, the risk is much greater for people over 60. how glaucoma develops  there are several different types of glaucoma. most of these involve the drainage system within the eye. at the front of the eye there is a small space called the anterior chamber. a clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (watch the video to learn more about glaucoma. to enlarge the video, click the brackets in the lower right-hand corner. to reduce the video, press the escape (esc) button on your keyboard.) in glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. as the fluid builds up, the pressure inside the eye rises. unless this pressure is controlled, it may cause damage to the optic n

# Ingest data to an SQL Table


In [21]:
%pip install --upgrade --quiet  langchain-google-cloud-sql-pg langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.14 requires openai<2.0.0,>=1.58.1, but you have openai 0.28.0 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from dotenv import load_dotenv
load_dotenv()

True

se connecter à une base de données PostgreSQL hébergée dans une instance Cloud SQL 

In [23]:
# Configuration de la connexion
engine = PostgresEngine.from_instance(
    project_id="my-dproject-452220",  # ID de votre projet
    instance="elyessiki",             # Nom de votre instance Cloud SQL
    region="europe-west1",            # Région de votre instance Cloud SQL
    database="health_database",             # Nom de la base de données (à adapter si nécessaire)
    user="postgres",                  # Nom d'utilisateur pour la connexion à PostgreSQL
    password=DB_PASSWORD              # Mot de passe (défini dans votre environnement)
)


In [42]:
from google.cloud import aiplatform
import logging

# Spécifiez votre projet et la région
project_id = "my-dproject-452220"
location = "europe-west1"  # Changez selon la région de votre projet

# Initialiser le client Vertex AI
try:
    aiplatform.init(project=project_id, location=location)
    print(f"Client Vertex AI initialisé avec succès pour le projet: {project_id} dans la région: {location}")
except Exception as e:
    logging.error(f"Erreur lors de l'initialisation de Vertex AI: {e}")

# Lister les modèles disponibles
try:
    models = aiplatform.Model.list()
    if not models:
        print("Aucun modèle trouvé.")
    else:
        # Afficher les modèles
        for model in models:
            print(f"Model ID: {model.resource_name}, Model Display Name: {model.display_name}")
except Exception as e:
    logging.error(f"Erreur lors de la récupération des modèles : {e}")


Client Vertex AI initialisé avec succès pour le projet: my-dproject-452220 dans la région: europe-west1
Aucun modèle trouvé.


export GOOGLE_APPLICATION_CREDENTIALS="my-dproject-452220-4778fcd52a8d.json"


In [37]:
import os
from google.cloud import aiplatform

# Définir la variable d'environnement pour la clé JSON
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my-dproject-452220-4778fcd52a8d.json"  # Remplacez par le chemin réel

# Initialiser le client Vertex AI
aiplatform.init(project="my-dproject-452220", location="europe-west1")
models = aiplatform.Model.list()
for model in models:
    print(f"Model ID: {model.resource_name}, Model Display Name: {model.display_name}")


In [39]:
from google.cloud import aiplatform

# Remplace avec ton projet et ta région
project_id = "my-dproject-452220"
location = "europe-west1"  # Certains modèles sont uniquement disponibles dans certaines régions

# Initialiser Vertex AI
aiplatform.init(project=project_id, location=location)

# Lister les modèles disponibles
models = aiplatform.Model.list()

# Afficher les modèles
for model in models:
    print(f"Model ID: {model.resource_name}, Display Name: {model.display_name}")


In [43]:
import os
from google.cloud import aiplatform

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my-dproject-452220-4778fcd52a8d.json"
os.environ["CLOUDSDK_CORE_PROJECT"] = "my-dproject-452220"  # Force le bon projet

aiplatform.init(project="my-dproject-452220", location="europe-west1")

models = aiplatform.Model.list(filter="display_name:embedding")

if models:
    for model in models:
        print(f"ID: {model.resource_name}, Nom: {model.display_name}")
else:
    print("❌ Aucun modèle trouvé. Vérifie la région et l'API activée.")


InvalidArgument: 400 Provided filter is not valid.

In [38]:
embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest",
    project="my-dproject-452220"
)

In [46]:
import nest_asyncio
nest_asyncio.apply()

async def init_db():
    try:
        # Utiliser la table existante
        vector_store = PostgresVectorStore.create_sync(
            engine=engine,
            table_name="elyes_med",
            embedding_service=embeddings,
        )
        print("Vector store créé avec succès")
        return vector_store
    except Exception as e:
        print(f"Error: {e}")
        return None

# Création du vector store uniquement
vector_store = await init_db()

Vector store créé avec succès


In [47]:
'''
print(f"Nombre total de documents : {len(documents)}")

# Vérifier s'il y a des NaN dans les documents
nan_count = 0
for doc in documents:
    if 'NaN' in str(doc.metadata) or 'nan' in str(doc.metadata):
        nan_count += 1
print(f"\nNombre de documents contenant NaN : {nan_count}")

# Vérifier les remplacements
reponse_manquante = sum(1 for doc in documents if doc.metadata['answer'] == 'Pas de réponse disponible')
focus_non_cat = sum(1 for doc in documents if doc.metadata['focus_area'] == 'Non catégorisé')

print(f"\nNombre de 'Pas de réponse disponible' : {reponse_manquante}")
print(f"Nombre de 'Non catégorisé' : {focus_non_cat}")

# Afficher quelques exemples
print("\nExemples de documents avec valeurs remplacées :")
for doc in documents[:5]:
    print("\nQuestion:", doc.page_content)
    print("Métadonnées:", doc.metadata)
    '''

'\nprint(f"Nombre total de documents : {len(documents)}")\n\n# Vérifier s\'il y a des NaN dans les documents\nnan_count = 0\nfor doc in documents:\n    if \'NaN\' in str(doc.metadata) or \'nan\' in str(doc.metadata):\n        nan_count += 1\nprint(f"\nNombre de documents contenant NaN : {nan_count}")\n\n# Vérifier les remplacements\nreponse_manquante = sum(1 for doc in documents if doc.metadata[\'answer\'] == \'Pas de réponse disponible\')\nfocus_non_cat = sum(1 for doc in documents if doc.metadata[\'focus_area\'] == \'Non catégorisé\')\n\nprint(f"\nNombre de \'Pas de réponse disponible\' : {reponse_manquante}")\nprint(f"Nombre de \'Non catégorisé\' : {focus_non_cat}")\n\n# Afficher quelques exemples\nprint("\nExemples de documents avec valeurs remplacées :")\nfor doc in documents[:5]:\n    print("\nQuestion:", doc.page_content)\n    print("Métadonnées:", doc.metadata)\n    '

In [17]:
'''print(f"Nombre total de documents : {len(documents)}")

# Vérifier chaque champ séparément
nan_in_answer = 0
nan_in_source = 0
nan_in_focus = 0

for doc in documents:
    if pd.isna(doc.metadata['answer']) or doc.metadata['answer'] == 'NaN':
        nan_in_answer += 1
    if pd.isna(doc.metadata['source']) or doc.metadata['source'] == 'NaN':
        nan_in_source += 1
    if pd.isna(doc.metadata['focus_area']) or doc.metadata['focus_area'] == 'NaN':
        nan_in_focus += 1

print("\nAnalyse des NaN par champ :")
print(f"NaN dans answer : {nan_in_answer}")
print(f"NaN dans source : {nan_in_source}")
print(f"NaN dans focus_area : {nan_in_focus}")

print(f"\nValeurs remplacées :")
print(f"'Pas de réponse disponible' : {sum(1 for doc in documents if doc.metadata['answer'] == 'Pas de réponse disponible')}")
print(f"'Non catégorisé' : {sum(1 for doc in documents if doc.metadata['focus_area'] == 'Non catégorisé')}")

# Afficher un exemple de document avec valeur remplacée
print("\nExemple de document avec 'Pas de réponse disponible' ou 'Non catégorisé' :")
for doc in documents:
    if doc.metadata['answer'] == 'Pas de réponse disponible' or doc.metadata['focus_area'] == 'Non catégorisé':
        print("\nQuestion:", doc.page_content)
        print("Métadonnées:", doc.metadata)
        break'''

'print(f"Nombre total de documents : {len(documents)}")\n\n# Vérifier chaque champ séparément\nnan_in_answer = 0\nnan_in_source = 0\nnan_in_focus = 0\n\nfor doc in documents:\n    if pd.isna(doc.metadata[\'answer\']) or doc.metadata[\'answer\'] == \'NaN\':\n        nan_in_answer += 1\n    if pd.isna(doc.metadata[\'source\']) or doc.metadata[\'source\'] == \'NaN\':\n        nan_in_source += 1\n    if pd.isna(doc.metadata[\'focus_area\']) or doc.metadata[\'focus_area\'] == \'NaN\':\n        nan_in_focus += 1\n\nprint("\nAnalyse des NaN par champ :")\nprint(f"NaN dans answer : {nan_in_answer}")\nprint(f"NaN dans source : {nan_in_source}")\nprint(f"NaN dans focus_area : {nan_in_focus}")\n\nprint(f"\nValeurs remplacées :")\nprint(f"\'Pas de réponse disponible\' : {sum(1 for doc in documents if doc.metadata[\'answer\'] == \'Pas de réponse disponible\')}")\nprint(f"\'Non catégorisé\' : {sum(1 for doc in documents if doc.metadata[\'focus_area\'] == \'Non catégorisé\')}")\n\n# Afficher un exemp

In [ ]:
#premiere execution seulement pour remplir la table avec les documents

#vector_store.add_documents(documents)

['ce590b4f-4e4c-424a-826a-0c442ae14f99',
 'f59ff6db-b5b6-490d-84a3-e1bf90145d37',
 '6769268f-cc1c-4e4f-9a3c-71f118640af9',
 '4e628a99-0bd5-4e89-8f00-6e5541d98ff7',
 '6b72358a-f3b1-4028-8ebd-a69125fce6df',
 '28b2a736-70bd-4c63-b00b-c6a92b5cbaa9',
 '3d15a14b-ad26-46d2-be14-119b83f8d257',
 '72722850-d658-46ef-8b0a-d59ca8fd4ea9',
 '0c815ac0-0321-49bf-9702-fc0a44243bf7',
 '3e9a0ebe-8f38-4ee9-b112-ca1433be090d',
 'da29d74e-1aec-4640-a800-62670b04c538',
 'ff205f50-367d-4f34-b0bf-3a8198ec3424',
 '61fccb1b-12e9-4883-b9cd-228ce925e8ca',
 '3813b41f-11c6-4853-8c09-9f5b1dcc7b38',
 '0846c968-fb94-4499-8d33-56b9499261f2',
 '90b0e390-f4d4-4280-a8f1-3623e27b0f18',
 'e4c6a646-7e77-474e-a4c7-1315a01537fc',
 'c2e94986-e7b3-4658-b896-320774c53115',
 '49ee0606-15da-4ade-8e28-87e7647f2e47',
 '56d9dc84-5368-46b9-bd67-c9fedc6055ea',
 '7639a152-ffec-44b4-ab4c-96bdc63b0903',
 'd0325cb6-a1ad-4bb1-a761-8a5eacd068ca',
 '91fc2420-de98-4359-82f1-b73cc96ce058',
 'bd9ea29b-6221-450f-b769-52c77c3fbec5',
 'd909a824-77d9-

In [48]:
vector_store.add_documents(documents)

['2ac3bedd-780b-4cf3-9105-68a27f2d563f',
 'd9cbf7bc-56b8-45eb-9b59-afcb63929b5a',
 '7e0018a3-8c4c-4b4d-9cd6-130ea6a5bb9d',
 '9ebd8057-7d05-44f0-8821-7996825b7b42',
 'a973ee0c-be13-4fa2-8c76-e04a11936049',
 '05ce6dd4-2e9f-4096-a865-009553839824',
 'fd0a0535-bb76-4a66-a1df-b4fbe3f68d5b',
 '985453c3-da01-4071-a26b-f20a2f36cf99',
 '70d94278-9161-4b25-808b-fc004ae6d6a6',
 'eaec6ec9-5ff4-456f-aa6a-984affbb0583',
 '130c3521-d8a0-49b8-99ca-8ac2531099bd',
 'd9d9a117-97cb-4023-99c6-16095ab90f98',
 '32bf32db-7ff1-4610-a21e-657deab1b4e8',
 '43116708-1d38-4dc3-aa7f-d8206c87d1a5',
 '424657da-5063-4d69-8b32-a34f64fec2de',
 'aaff1f3b-950a-423d-8f37-19987e84bdd2',
 'bee3b9e8-44d0-4d86-acc4-bbc5e4b9478e',
 '01f509dc-0c81-4421-97e0-dbcccf14276a',
 '4ceb047f-9dc7-4810-8257-770589170587',
 'f403e2ad-b68a-4750-876b-b880b4bcd98f',
 '59663eea-28da-41b7-8ca9-275ebe99ce3f',
 '101c8227-af28-4702-8309-3de4b5822e28',
 '417e3562-2829-48d9-9903-96973a59187b',
 '30c16e82-fd89-444d-ac1f-30469996a454',
 'e45a5153-aee7-

# Query search


In [49]:
query = "what is Glaucoma  ?"

In [50]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)

docs = retriever.invoke(query)

In [ ]:
results = vector_store.similarity_search_with_score(query, k=5)
#vector_store.similarity_search_with_relevance_scores(query=query, k=3)

print(f"Found {len(results)} documents\n")

for doc, score in results:
    print(f"Score: {score}")
    print("Question:", doc.page_content)
    print("Answer:", doc.metadata['answer'])
    print("Source:", doc.metadata['source'])
    #print("Focus Area:", doc.metadata['focus_area'])
    print("\n---\n")
    

Found 5 documents

Score: 0.07265930211533544
Question: what is (are) glaucoma ?
Answer: the optic nerve is a bundle of more than 1 million nerve fibers. it connects the retina to the brain.
Source: nihseniorhealth

---

Score: 0.07265930211533544
Question: what is (are) glaucoma ?
Answer: open-angle glaucoma is the most common form of glaucoma. in the normal eye, the clear fluid leaves the anterior chamber at the open angle where the cornea and iris meet. when the fluid reaches the angle, it flows through a spongy meshwork, like a drain, and leaves the eye. sometimes, when the fluid reaches the angle, it passes too slowly through the meshwork drain, causing the pressure inside the eye to build. if the pressure damages the optic nerve, open-angle glaucoma -- and vision loss -- may result.
Source: nihseniorhealth

---

Score: 0.07265930211533544
Question: what is (are) glaucoma ?
Answer: glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and 